In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import time
    
# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [2]:
flujos_loc = pd.read_csv('tablas/dd_localidades.csv')
print(flujos_loc.shape[0])
flujos_loc.head(3)

377610


,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.0,583715
1,10202521,1020,2521,1304729,12200,90.0,612802
2,10202522,1020,2522,1304729,2659,17.0,617076


In [3]:
locs = cargar_datos_geo()[2][['CODLOC', 'geometry']]
locs.head(3)

,CODLOC,geometry
0,2220,POINT (549273.219 6633915.063)
1,2521,POINT (442454.671 6652273.541)
2,2522,POINT (457921.001 6633361.660)


In [4]:
locs_cod = list(locs.CODLOC)
len(locs_cod) ** 2 - len(locs_cod)

377610

In [5]:
# pega geometrías
dd = flujos_loc.merge(locs, left_on='cod_ori', right_on='CODLOC')
dd = dd.merge(locs, left_on='cod_des', right_on='CODLOC')
dd.drop(['CODLOC_x', 'CODLOC_y'], axis=1, inplace=True)
dd.rename({'geometry_x':'geom_ori', 'geometry_y':'geom_des'}, axis=1, inplace=True)

dd.head()

# exporta CSV con geometrías puntuales para mapa de flujos
#dd.to_csv('capas/loc_lines.csv', index=False)

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m,geom_ori,geom_des
0,10202220,1020,2220,1304729,40657,635.0,583715,POINT (573082.248 6145008.434),POINT (549273.219 6633915.063)
1,25212220,2521,2220,12200,40657,NaN,137927,POINT (442454.671 6652273.541),POINT (549273.219 6633915.063)
2,25222220,2522,2220,2659,40657,NaN,111408,POINT (457921.001 6633361.660),POINT (549273.219 6633915.063)
3,26212220,2621,2220,2531,40657,NaN,106867,POINT (468458.270 6601361.867),POINT (549273.219 6633915.063)
4,27212220,2721,2220,380,40657,NaN,58392,POINT (502880.928 6648162.053),POINT (549273.219 6633915.063)


In [6]:
print(flujos_loc.shape)
flujos_loc.head()

(377610, 7)


,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.0,583715
1,10202521,1020,2521,1304729,12200,90.0,612802
2,10202522,1020,2522,1304729,2659,17.0,617076
3,10202621,1020,2621,1304729,2531,9.0,595785
4,10202721,1020,2721,1304729,380,NaN,628213


In [7]:
# reemplaza ceros
def nozeros(df):
    df_copy = df
    df_copy.loc[df_copy.personas_mig.isna(), 'personas_mig'] = 0.001
    df_copy.loc[df_copy.poblacion_des==0, 'poblacion_des'] = 1
    df_copy.loc[df_copy.poblacion_ori==0, 'poblacion_ori'] = 1
    return df

In [8]:
flujos_loc = nozeros(flujos_loc)
flujos_loc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.000,583715
1,10202521,1020,2521,1304729,12200,90.000,612802
2,10202522,1020,2522,1304729,2659,17.000,617076
3,10202621,1020,2621,1304729,2531,9.000,595785
4,10202721,1020,2721,1304729,380,0.001,628213


In [9]:
# convierte códigos a string para ser correctamente interpretados por el modelo
flujos_loc['cod_ori'] = flujos_loc['cod_ori'].astype(str)
flujos_loc['cod_des'] = flujos_loc['cod_des'].astype(str)

flujos_loc_oc = flujos_loc

flujos_loc_oc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.000,583715
1,10202521,1020,2521,1304729,12200,90.000,612802
2,10202522,1020,2522,1304729,2659,17.000,617076
3,10202621,1020,2621,1304729,2531,9.000,595785
4,10202721,1020,2721,1304729,380,0.001,628213


In [ ]:
#zero inflated
from patsy import dmatrices


df = flujos_loc

expr = "personas_mig ~ cod_ori + log_dist -1"

mask = np.random.rand(len(df)) < 0.8
df_train = df[mask]
df_test = df[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))


y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')

zip_training_results = sm.ZeroInflatedPoisson(endog=y_train, exog=X_train, exog_infl=X_train, inflation='logit').fit()

zip_training_results.summary()

zip_predictions = zip_training_results.predict(X_test,exog_infl=X_test)

predicted_counts=np.round(zip_predictions)

actual_counts = y_test['personas_mig']

print('ZIP RMSE='+str(np.sqrt(np.sum(np.power(np.subtract(predicted_counts,actual_counts),2)))))



fig = plt.figure(figsize=(12, 8), dpi=150)

fig.suptitle('Predicted versus actual counts using the ZIP model')

predicted, = plt.plot(X_test.index, predicted_counts, 'g', label='Predicted')

actual, = plt.plot(X_test.index, actual_counts, 'r', label='Actual')

plt.legend(handles=[predicted, actual])

plt.show()